### 아이디어
- 1912(연속합)를 10만번 푸는 문제이다. 
- 위 문제를 활용하고자 한다면, $acc$ 이 $default$ 로 초기화 한다는 것은, 그 이전까지의 누적합을 더하는 것이 의미없다는 것을 의미한다.
  - 길이 $N$ 인 수열의 최대 연속 부분 누적합을 구할 수는 있겠는데, 시작점이 0이 아닌 경우에 대해선 아이디어가 필요하다.
  - 초기화해야 하는 지점을 $O(\log N)$ 만에 찾을 수 있지 않을까?

### 풀이
- 빨리 보길 잘한거 같다. 자력으로 절대 떠올릴 수 없을 법한 세그먼트의 사용법이 사용됐다.
- https://wbjeon2k.github.io/ps/almight/2021/09/19/BOJ16993.html
- 구간합 $s$, 왼쪽 끝 점을 포함할 때의 구간합 최대 $ls$, 오른쪽 끝 점을 포함할 때의 구간합 최대 $rs$, 구간합 최대 $ms$ 라고하고, 각각의 자식노드를 $L, R$ 이라고 할때
  - $ls = \max(ls, L_s + R_{ls})$
  - $rs = \max(rs, R_s + L_{rs})$
  - $ms = \max(ls, rs, L_{rs} + R_{ls})$
  - $s = L_s + R_s$
    - 구간합을 구할 때는 자식노드의 구간합을 더하면 된다.
- 분할 정복을 통해 가장 왼쪽/오른쪽의 원소를 포함한 최대 부분합을 구하고, 쿼리를 수행할 때 각 필요한 구간을 합치는 테크닉을 요구한다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = 10 ** 18

from dataclasses import dataclass
@dataclass
class Node :
  ls: int = -INF
  rs: int = -INF
  ms: int = -INF
  s: int = 0

class ST :
  def __init__(self, L):
    self._size = _size = 1 << (len(L) - 1).bit_length()
    self.L = L
    self.seg = [Node() for _ in range(2 * _size)]
    self.init(1, len(L), 1)

  def init(self, s, e, x) :
    if e == s :
      self.seg[x] = Node(self.L[s - 1], self.L[s - 1], self.L[s - 1], self.L[s - 1])
      return self.seg[x]

    m = (s + e) // 2
    L = self.init(s, m, x * 2)
    R = self.init(m + 1, e, x * 2 + 1)
    
    self.seg[x].ls = max(L.ls, L.s + R.ls)
    self.seg[x].rs = max(R.rs, R.s + L.rs)
    self.seg[x].ms = max(L.ms, R.ms, L.rs + R.ls)
    self.seg[x].s = L.s + R.s
    return self.seg[x]

  def _query(self, l, r, s, e, x) :
    if l <= s and e <= r :
      return self.seg[x]
    if r < s or e < l :
      return Node()
    m = (s + e) // 2
    L = self._query(l, r, s, m, x * 2)
    R = self._query(l, r, m + 1, e, x * 2 + 1)

    ret = Node()
    ret.ls = max(L.ls, L.s + R.ls)
    ret.rs = max(R.rs, R.s + L.rs)
    ret.ms = max(L.ms, R.ms, L.rs + R.ls)
    ret.s = L.s + R.s
    return ret

  def query(self, s, e):
    return self._query(s, e, 1, len(self.L), 1)

def sol():
  N = int(input())
  L = list(map(int, input().split()))
  st = ST(L)

  ans = []
  M = int(input())
  for i in range(M):
    s, e = map(int, input().split())
    ans.append(st.query(s, e).ms)
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

- iterative 솔루션

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = 10 ** 18

from dataclasses import dataclass
@dataclass
class Node :
  ls: int = -INF
  rs: int = -INF
  ms: int = -INF
  s: int = 0

class ST : 
  def __init__(self, L): #default는 Node()이다.
    self._size = _size = 1 << (len(L) - 1).bit_length()
    self.L = L
    self.seg = [Node() for _ in range(2 * _size)]
    self.seg[_size:_size + len(L)] = [Node(s = x, ls = x, rs = x, ms = x) for x in L]
    self.init()

  def init(self) :
    for i in reversed(range(self._size)) :
      L = self.seg[i * 2]
      R = self.seg[i * 2 + 1]
      
      self.seg[i].ls = max(L.ls, L.s + R.ls)
      self.seg[i].rs = max(R.rs, R.s + L.rs)
      self.seg[i].ms = max(L.ms, R.ms, L.rs + R.ls)
      self.seg[i].s = L.s + R.s

  def merge(self, l, r):
    ret = Node()
    ret.ls = max(l.ls, l.s + r.ls)
    ret.rs = max(r.rs, r.s + l.rs)
    ret.ms = max(l.ms, r.ms, l.rs + r.ls)
    ret.s = l.s + r.s
    return ret

  def query(self, s, e): #[s, e]
    s += self._size - 1
    e += self._size

    l = Node()
    r = Node()
    while s < e :
      if s & 1 :
        l = self.merge(l, self.seg[s]) #필요한 구간에 대해 결과값을 병합해준다.
        s += 1
      if e & 1 :
        e -= 1
        r = self.merge(self.seg[e], r)
      s >>= 1
      e >>= 1
    return self.merge(l, r)

def sol():
  N = int(input())
  L = list(map(int, input().split()))
  st = ST(L)

  ans = []
  M = int(input())
  for _ in range(M):
    s, e = map(int, input().split())
    ans.append(st.query(s, e).ms)
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()